# Pretraining Training script in MAE

> Pretraining Training script in MAE

In [ ]:
#| default_exp mae.pretraining_training

In [1]:
#| hide
%load_ext autoreload
%autoreload 2

In [41]:
#| export
import argparse
import datetime
import json
import numpy as np
from typing import Iterable
import os
import time
from pathlib import Path

import torch
import torch.backends.cudnn as cudnn
from torch.utils.tensorboard import SummaryWriter
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import timm

In [3]:
timm.__version__

'1.0.9'

In [5]:
#| exporti
import segmentation_test.mae.misc as misc

In [11]:
#| export
import timm.optim.optim_factory as optim_factory
import math
from fastcore.test import *

### Gradient Scaling and Norm Counting: 


Dealing with mixed precision training, where we use both 16-bit and 32-bit floating-point numbers to speed up computation and reduce memory usage. But this can lead to some tricky numerical issues. Enter the `NativeScalerWithGradNormCount` class!

## What's this class all about?

This nifty little class is a wrapper around PyTorch's `GradScaler`. It's designed to handle the intricacies of mixed precision training while also giving us some extra goodies like gradient norm calculation.

## Let's break it down:

1. **Initialization**: We start by creating a `GradScaler` object. This is PyTorch's built-in tool for automatic mixed precision training.

2. **The `__call__` method**: This is where the magic happens!
   - It scales the loss and performs backpropagation.
   - If we're updating gradients, it handles gradient unscaling and clipping.
   - It also calculates the gradient norm, which is super useful for monitoring training stability.

3. **State management**: The `state_dict` and `load_state_dict` methods allow us to save and load the scaler's state. This is crucial for resuming training from checkpoints.

## Why is this so cool?

- It seamlessly integrates mixed precision training into our workflow.
- It provides gradient clipping out of the box, which helps prevent exploding gradients.
- The gradient norm calculation gives us valuable insights into our training process.

By using this class, we're not just training our model - we're training it smartly and efficiently. It's like having a personal trainer for your neural network!


In [22]:
#| export
class NativeScalerWithGradNormCount:
    state_dict_key = "amp_scaler"

    def __init__(self):
        self._scaler = torch.cuda.amp.GradScaler()

    def __call__(
		self, 
		loss, 
		optimizer, 
		clip_grad=None, 
		parameters=None, 
		create_graph=False, 
		update_grad=True):
        self._scaler.scale(loss).backward(create_graph=create_graph)
        if update_grad:
            if clip_grad is not None:
                assert parameters is not None
                self._scaler.unscale_(optimizer)  # unscale the gradients of optimizer's assigned params in-place
                norm = torch.nn.utils.clip_grad_norm_(parameters, clip_grad)
            else:
                self._scaler.unscale_(optimizer)
                norm = get_grad_norm_(parameters)
            self._scaler.step(optimizer)
            self._scaler.update()
        else:
            norm = None
        return norm

    def state_dict(self):
        return self._scaler.state_dict()

    def load_state_dict(self, state_dict):
        self._scaler.load_state_dict(state_dict)

In [9]:
#| export
def adjust_learning_rate(
	optimizer, 
	epoch, 
	args
	):
    """Decay the learning rate with half-cycle cosine after warmup"""
    if epoch < args.warmup_epochs:
        lr = args.lr * epoch / args.warmup_epochs 
    else:
        lr = args.min_lr + (args.lr - args.min_lr) * 0.5 * \
            (1. + math.cos(math.pi * (epoch - args.warmup_epochs) / (args.epochs - args.warmup_epochs)))
    for param_group in optimizer.param_groups:
        if "lr_scale" in param_group:
            param_group["lr"] = lr * param_group["lr_scale"]
        else:
            param_group["lr"] = lr
    return lr


In [21]:
#| export
def test_adjust_learning_rate():
    class DummyOptimizer:
        def __init__(self):
            self.param_groups = [
				{"lr": 0.1}, 
				{"lr": 0.2, "lr_scale": 2}
			]

    class DummyArgs:
        def __init__(self):
            self.warmup_epochs = 5
            self.epochs = 100
            self.lr = 0.1
            self.min_lr = 0.001

    optimizer = DummyOptimizer()
    args = DummyArgs()

    # Test during warmup
    lr = adjust_learning_rate(optimizer, 2, args)
    test_eq(lr, 0.04)
    test_eq(optimizer.param_groups[0]["lr"], 0.04)
    test_eq(optimizer.param_groups[1]["lr"], 0.08)

    # Test after warmup
    lr = adjust_learning_rate(optimizer, 50, args)
    expected_lr = 0.001 + (0.1 - 0.001) * 0.5 * (1 + math.cos(math.pi * 45 / 95))
    test_close(lr, expected_lr, eps=1e-6)
    test_close(optimizer.param_groups[0]["lr"], expected_lr, eps=1e-6)
    test_close(optimizer.param_groups[1]["lr"], expected_lr * 2, eps=1e-6)
    print("All tests passed!")


In [20]:
test_adjust_learning_rate()

All tests passed!


In [45]:
#| export
def train_one_epoch(
	            model: torch.nn.Module,
                data_loader: Iterable, 
				optimizer: torch.optim.Optimizer,
                device: torch.device, 
				epoch: int, 
				loss_scaler: NativeScalerWithGradNormCount,
                log_writer=None,
                args=None):
    model.train(True)
    metric_logger = misc.MetricLogger(delimiter="  ")
    metric_logger.add_meter('lr', misc.SmoothedValue(window_size=1, fmt='{value:.6f}'))
    header = 'Epoch: [{}]'.format(epoch)
    print_freq = 20

    accum_iter = args.accum_iter

    optimizer.zero_grad()

    if log_writer is not None:
        print('log_dir: {}'.format(log_writer.log_dir))

    for data_iter_step, (samples, _) in enumerate(metric_logger.log_every(data_loader, print_freq, header)):

        # we use a per iteration (instead of per epoch) lr scheduler
        if data_iter_step % accum_iter == 0:
            lr_sched.adjust_learning_rate(optimizer, data_iter_step / len(data_loader) + epoch, args)

        samples = samples.to(device, non_blocking=True)

        with torch.cuda.amp.autocast():
            loss, _, _ = model(samples, mask_ratio=args.mask_ratio)

        loss_value = loss.item()

        if not math.isfinite(loss_value):
            print("Loss is {}, stopping training".format(loss_value))
            sys.exit(1)

        loss /= accum_iter
        loss_scaler(
			loss, 
			optimizer, 
			parameters=model.parameters(),
            update_grad=(data_iter_step + 1) % accum_iter == 0
		)
        if (data_iter_step + 1) % accum_iter == 0:
        	optimizer.zero_grad()
        torch.cuda.synchronize()

        metric_logger.update(loss=loss_value)

        lr = optimizer.param_groups[0]["lr"]
        metric_logger.update(lr=lr)

        loss_value_reduce = misc.all_reduce_mean(loss_value)
        if log_writer is not None and (data_iter_step + 1) % accum_iter == 0:
            """ We use epoch_1000x as the x-axis in tensorboard.
			This calibrates different curves when batch size changes.
			"""
            epoch_1000x = int((data_iter_step / len(data_loader) + epoch) * 1000)
            log_writer.add_scalar('train_loss', loss_value_reduce, epoch_1000x)
            log_writer.add_scalar('lr', lr, epoch_1000x)


	# gather the stats from all processes
    metric_logger.synchronize_between_processes()
    print("Averaged stats:", metric_logger)
    return {k: meter.global_avg for k, meter in metric_logger.meters.items()}

# main_pretrain.py script will be implemented here

> engine_pretrain.py needs to be implemented here, and call main function